# Day 3: Supply II - Enhancing the model

We will continue our modeling exercise by adding carbon taxes, renewable subsidies, and investment to the model.

This will allow us to consider the policy impacts of alternative energy transition policies.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [2]:
#using Pkg
#Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Plots", "Printf"])


In [1]:
using DataFrames
using CSV
using JuMP
using Ipopt
using Plots
using Printf

Remember to set your path correctly:

## Building the model

We load the same data as last week, and also clean it up to simplify it further and create the demand and import curves.

In [7]:
dfclust = CSV.read("data_jaere_clustered.csv", DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate demand based on elasticities (using 0.1 here as only one final demand)
elas = [.1, .2, .5, .3];
dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

# Calibrate imports (using elas 0.3)
dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [5]:
dfclust

,price,imports,q_commercial,q_industrial,q_residential,wind_cap,solar_cap
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,44.8507,7.25688,15.6342,6.24111,11.3666,0.481806,0.785586
2,40.2834,8.47216,11.5798,4.06036,14.155,0.411881,0.00733263
3,70.1554,8.07499,15.0325,6.11547,20.0267,0.398612,0.505725
4,26.5618,7.02309,10.0495,3.91292,8.15435,0.662615,0.0102827
5,37.0007,5.6396,10.0141,3.54231,10.0543,0.208139,0.450361
6,37.998,6.61716,9.33798,3.20469,8.74743,0.141276,0.0105646
7,43.3107,9.1167,19.8993,7.09356,5.91624,0.232668,0.577826
8,32.181,8.7686,14.153,3.59615,9.66465,0.192524,0.112814
9,44.9164,6.89629,15.5495,5.22585,6.4413,0.209247,0.0267234


The technology file now includes the fixed cost of building new power plants (technologies 3-5). Note that we added an additional row for new natural gas plants.

We will use an annualization factor to pro-rate the importance of fixed costs for one year.

In [3]:
tech = CSV.read("data_technology.csv", DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;
tech.F2 = tech.F2 ./afactor;
tech



Row,techname,heatrate,heatrate2,F,capLB,capUB,new,renewable,solar,thermal,e,e2,c,c2,F2
,String15,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,Hydro/Nuclear,10.0,0.0,0.0,1.0,1.0,0,0,0,0,0.0,0.0,10.0,0.0,0.0
2,Existing 1,6.67199,0.0929123,0.0,11.5,11.5,0,0,0,1,0.360184,0.0048861,23.352,0.325193,0.0
3,Existing 2,9.79412,0.286247,0.0,14.5,14.5,0,0,0,1,0.546134,0.0110777,34.2794,1.00187,0.0
4,Existing 3,13.8181,20.5352,0.0,0.578,0.578,0,0,0,1,0.816768,0.234476,48.3634,71.8731,0.0
5,New Gas,6.6,0.0,78.4773,0.0,100.0,1,0,0,1,0.35,0.0,23.352,0.325193,1.56955
6,Wind,0.0,0.0,100.303,0.0,100.0,1,1,0,0,0.0,0.0,0.0,0.0,2.00606
7,Solar,0.0,0.0,100.303,0.0,100.0,1,1,1,0,0.0,0.0,0.0,0.0,2.00606


### Adding investment

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$$ \max \ CS - Costs - Fixed Costs \\

\text{s.t.} \ \text{operational constraints, market clearing}. $$

Notice that we added the fixed costs to the problem, as we will be solving for the optimal level of wind and solar investment.

In [44]:
## Clear market based on cost minimization
function clear_market_invest(data::DataFrame, tech::DataFrame)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    
    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);
    
    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                        (gross_surplus[t] - costs[t] ) for t=1:T)
                    - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
                    - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);

    
    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest (generic function with 1 method)

In [45]:
results_min = clear_market_invest(dfclust, tech)

Dict{String, Any} with 10 entries:
  "avg_price" => 33.5045
  "solar_gw"  => -9.97587e-9
  "cost"      => [740.963, 573.792, 1071.45, 350.522, 457.893, 379.907, 657.347…
  "price"     => [42.4835, 38.4386, 61.2367, 26.5039, 36.0861, 27.6725, 40.2351…
  "gas_gw"    => 1.97069
  "status"    => "LOCALLY_SOLVED"
  "imports"   => [7.14198, 8.35576, 7.76702, 7.01849, 5.59778, 6.07772, 8.92248…
  "demand"    => [33.4174, 29.9316, 41.6982, 22.1216, 23.6691, 21.8686, 33.1428…
  "quantity"  => [4.61597 11.5 … 4.96219e-9 1.8045e-9; 3.95377 11.5 … 5.82e-9 9…
  "wind_gw"   => -9.69727e-9

## Environmental regulation

We will begin by adding carbon taxes to the model.



In [48]:
function clear_market_invest_tax(data::DataFrame, tech::DataFrame,tax=30.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    # New variable
    @variable(model, totale[1:T]);

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - tax*totale[t]) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] + tech.e2[i] * quantity[t,i]^2 for i=1:I))

        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);

    
    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_tax (generic function with 2 methods)

In [51]:
results_tax = clear_market_invest_tax(dfclust, tech)

Dict{String, Any} with 12 entries:
  "avg_price"     => 43.6926
  "price"         => [57.2074, 51.199, 70.3597, 36.9793, 43.5261, 40.0505, 55.2…
  "gas_gw"        => 1.84275
  "status"        => "LOCALLY_SOLVED"
  "totale"        => [7.74857, 5.60993, 13.4352, 2.39038, 5.43327, 4.5213, 7.01…
  "quantity"      => [4.61597 11.5 … 2.58407 1.76044e-9; 3.95377 11.5 … 2.20904…
  "solar_gw"      => -9.9378e-9
  "imports"       => [7.85668, 9.16087, 8.08204, 7.84942, 5.93798, 6.72439, 9.8…
  "demand"        => [32.3261, 28.9878, 41.1627, 21.2493, 23.1943, 21.1751, 32.…
  "avg_emissions" => 4.77937
  "cost"          => [601.551, 466.937, 952.03, 242.169, 404.922, 345.999, 575.…
  "wind_gw"       => 5.36329

Finally, we will add a subsidy to the model. One can think of the subsidy as a negative cost to renewable power. 

In [52]:
function clear_market_invest_subsidy(data::DataFrame, tech::DataFrame,subsidy=10.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    # New variable
    @variable(model, totale[1:T]);

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 
                    - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
        sum(tech.e[i] * quantity[t,i] + tech.e2[i] * quantity[t,i]^2 for i=1:I))


        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_subsidy (generic function with 2 methods)

In [55]:
results_subsidy = clear_market_invest_subsidy(dfclust, tech)

Dict{String, Any} with 12 entries:
  "avg_price"     => 32.7931
  "price"         => [41.3537, 37.6123, 53.0627, 25.8507, 36.1026, 30.7403, 40.…
  "gas_gw"        => 0.975417
  "status"        => "LOCALLY_SOLVED"
  "totale"        => [9.53837, 7.06908, 15.4321, 3.39742, 6.16026, 5.1297, 8.71…
  "quantity"      => [4.61597 11.5 … 2.2615 1.45904e-9; 3.95377 11.5 … 1.93328 …
  "solar_gw"      => -9.87458e-9
  "imports"       => [7.08714, 8.30363, 7.48477, 6.96668, 5.59854, 6.23799, 8.9…
  "demand"        => [33.5012, 29.9927, 42.1779, 22.176, 23.668, 21.6967, 33.14…
  "avg_emissions" => 5.83137
  "cost"          => [645.062, 497.396, 1006.39, 241.764, 425.028, 354.239, 619…
  "wind_gw"       => 4.69379

## Follow-up exercises

1. Consider a tax and a subsidy that reach the same target of emissions. What are the different costs? How are the different components of welfare affected?

Note: This will require you to include emissions as an input or an output to the function.